protocle init, eq, repr

In [11]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"<Vector:x={self.x}, y={self.y}>"

    def __eq__(self, other):
        return (self.x == other.x) and (self.y == other.y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

        
vector1 = Vector(3, 4)
vector2 = Vector(3, 4)
print(vector) #__repr__
print(vector1 == vector2)   #__eq__
print(vector1 + vector2) #__add__


<Vector : x =3, y=4>
True
<Vector:x=6, y=8>


In [ ]:
from dataclasses import dataclass

@dataclass
class VectorDataclass:
    x : int
    y : int
    z : int
    
    def __add__(self, other):
        return VectorDataclass(self.x + other.x, self.y + other.y, self.z + other.z) 
       
vector1 = VectorDataclass(z=3, y=4, x=5)
vector2 = VectorDataclass(3, 4, 8)
print(vector1) #__repr__
print(vector1 == vector2)   #__eq__
print(vector1 + vector2) #__add__


Dataclass en lecture seule grâce à frozen

In [23]:
from dataclasses import dataclass

@dataclass(frozen = True)
class VectorDataclassFrozen:
    x : int
    y : int
    z : int
    
    def __add__(self, other):
        return VectorDataclassFrozen(self.x + other.x, self.y + other.y, self.z + other.z) 
       
vector1 = VectorDataclassFrozen(z=3, y=4, x=5)
vector2 = VectorDataclassFrozen(3, 4, 8)

vector1.x = 1959355 #ligne interdite car on essaye de modifier un attribut de la class frozen


FrozenInstanceError: cannot assign to field 'x'

In [32]:
user = UserDict()
user["A"] = "01"
user["B"] = "02"
user["C"] = "03"
user.__setitem__("D", "03")
print("Keys :")
print(list(user.keys()))
print("Values :")
print(list(user.values()))
print("Tuples :")
print(list(user.items()))

Keys :
['A', 'B', 'C', 'D']
Values :
['01', '02', '03', '03']
Tuples :
[('A', '01'), ('B', '02'), ('C', '03'), ('D', '03')]


Exemples ajout de fonctionnalité à une classe de pyhton dict -> passer par le module collections

In [37]:
from collections import UserDict

class NumeroTelephone(ValueError): 
    pass
    
class anuaire(UserDict):
    def __setitem__(self, key, value):
        if value in self.values():
            raise NumeroTelephone(f"Already Present with {value} and my key i try to put is {key}")
        super().__setitem__(key, value)
            

an = anuaire()
an["A"] = "01"
an["B"] = "02"
an["C"] = "03"

try:
    an["D"] = "01" # not possible
except NumeroTelephone as t:
    print(f"{t}")
an["B"] = "04"
print(an.items)

Already Present with 01 and my key i try to put is D
ItemsView({'A': '01', 'B': '02', 'C': '03'})


MRO

In [ ]:
class StylePython2:
    pass
class StylePython2_Fille(StylePython2):
    pass

# le rendre comptable pour python 3
class StylePython2(object):
    pass
class StylePython2_Fille(StylePython2):
    pass

In [ ]:
class StylePython3:
    pass
class StylePython3_Fille(StylePython3):
    pass

In [41]:
class CommonBase:
    def method(self):
        print("CommonBase")

class Base1(CommonBase):
    pass
    
class Base2(CommonBase):
    def method(self):
        print("Base2")
        
class MyClass(Base1, Base2):
    pass

myclass = MyClass()
myclass.method()  #normalemment python 2 appelle ma méthode de CommonBase !

Base2


In [46]:
def L(klass):
    print([klass_.__name__ for klass_ in  klass.__mro__])
L(MyClass)

['MyClass', 'Base1', 'Base2', 'CommonBase', 'object']


Démonstration probleme super vs init avec python 3

In [66]:
class A:
    def __init__(self):
        print("A", end =", ")
        super().__init__()

class B:
    def __init__(self):
        print("B", end =", ")
        super().__init__()

class C(A, B):
    def __init__(self):
        print("C", end =", ")
        A.__init__(self)  #avoid using _ini__ avec la classe , appelle tout le temps init avec super()
        B.__init__(self)
        #super().__init__()
        

c = C()

print("\nMRO")
print(L(C))


C, A, B, B, 
MRO
['C', 'A', 'B', 'object']
None


Démonstration super

In [63]:
class Mama:
    def say(self):
        print(" do your homework")

class Sister(Mama):
    def say(self):
        #Mama.say(self)
        #super(Sister, self).say() #ancienne facon
        super().say()
        print(" and your room")
        
sister = Sister()
sister.say()


 do your homework
 and your room


In [65]:
super(sister.__class__, sister).say()

 do your homework


Héritage et arguments args , kwargs

In [73]:
class A:
    def __init__(self, *args, **kwargs):
        print("A", end =", ")
        super().__init__(*args, **kwargs)

class B:
    def __init__(self, name, *args, **kwargs):
        print("B", end =", ")
        self.name = name
        super().__init__(*args, **kwargs)

class C(A, B):
    def __init__(self, name):
        print("C", end =", ")
        super().__init__(name)
        

c = C("Classe B")

print("\nMRO")
print(L(C))


C, A, B, 
MRO
['C', 'A', 'B', 'object']
None


Attenion à ne pas oublier des super().__init__ !!!!

In [77]:
class A:
    def __init__(self, *args, **kwargs):
        print("A", end =", ")
        #super().__init__(*args, **kwargs)  # si on oublie cela le construction de B n'est pas appelé !!!! 

class B:
    def __init__(self, name, *args, **kwargs):
        print("B", end =", ")
        self.name = name
        super().__init__(*args, **kwargs)

class C(A, B):
    def __init__(self, name):
        print("C", end =", ")
        super().__init__(name)
        

c = C("Classe B")

print("\nMRO")
print(L(C))


C, A, 
MRO
['C', 'A', 'B', 'object']
None


In [93]:
class MyClass: 
    __value = 1
        
myclass = MyClass()
print(myclass.__dict__)
print(MyClass.__dict__)
#print(myclass._value)
"__value"
"_MyClass_  ///    __value"

{}
{'__module__': '__main__', '_MyClass__value': 1, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


'_MyClass_  ///    __value'